In [ ]:
"""
ADVANCED OPTIMIZATION
Building on Base_Model_V1 baseline to achieve higher performance

Strategy:
1. Aggressive parameter tuning around the winning formula
2. More trees with optimized learning rates
3. Advanced feature engineering additions
4. Ensemble of ultra high performing variants
"""

import pandas as pd
import numpy as np
import os
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = "/Users/benmartin/Downloads/air_pollution_dataset"
np.random.seed(42)

def proven_target_transformation(y_train):
    """Keep the proven double-log transformation"""
    return np.log1p(np.log1p(y_train))

def safe_inverse_transform(predictions):
    """Keep the proven inverse transform"""
    predictions = np.clip(predictions, -10, 10)
    step1 = np.expm1(predictions)
    step1 = np.clip(step1, -10, 50)
    step2 = np.expm1(step1)
    step2 = np.clip(step2, 0, 2000)
    return step2

def create_enhanced_features(df):
    """Enhanced feature set, add carefully selected improvements to proven base"""
    df = df.copy()
    
    # Handle missing values
    df['latitude'].fillna(df['latitude'].median(), inplace=True)
    df['longitude'].fillna(df['longitude'].median(), inplace=True)
    
    # === PROVEN BASE FEATURES ===
    # Core cyclical features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    # Geographic features
    df['lat_abs'] = np.abs(df['latitude'])
    df['lon_abs'] = np.abs(df['longitude'])
    df['distance_equator'] = np.abs(df['latitude'])
    df['distance_prime_meridian'] = np.abs(df['longitude'])
    df['lat_squared'] = df['latitude'] ** 2
    df['lon_squared'] = df['longitude'] ** 2
    df['distance_origin'] = np.sqrt(df['latitude']**2 + df['longitude']**2)
    
    # Time categories
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['is_weekday'] = (df['day_of_week'] < 5).astype(int)
    df['is_morning_rush'] = ((df['hour'] >= 7) & (df['hour'] <= 9)).astype(int)
    df['is_evening_rush'] = ((df['hour'] >= 17) & (df['hour'] <= 19)).astype(int)
    df['is_night'] = ((df['hour'] < 6) | (df['hour'] > 22)).astype(int)
    df['is_business_hours'] = ((df['hour'] >= 9) & (df['hour'] <= 17)).astype(int)
    
    # Seasonal features
    df['is_winter'] = df['month'].isin([12, 1, 2]).astype(int)
    df['is_spring'] = df['month'].isin([3, 4, 5]).astype(int)
    df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)
    df['is_fall'] = df['month'].isin([9, 10, 11]).astype(int)
    
    # Proven interactions
    df['lat_lon_interaction'] = df['latitude'] * df['longitude']
    df['weekend_hour'] = df['is_weekend'] * df['hour']
    df['season_hour'] = df['month'] * df['hour']
    df['lat_month'] = df['latitude'] * df['month']
    df['coord_sum'] = df['latitude'] + df['longitude']
    df['coord_diff'] = df['latitude'] - df['longitude']
    df['rush_hour_indicator'] = df['is_morning_rush'] + df['is_evening_rush']
    df['weekend_lat'] = df['is_weekend'] * df['latitude']
    df['rush_lat'] = df['rush_hour_indicator'] * df['latitude']
    
    # === CAREFULLY SELECTED ENHANCEMENTS ===
    # Additional cyclical harmonics (proven to help in time series)
    df['hour_sin_12'] = np.sin(4 * np.pi * df['hour'] / 24)  # 12-hour cycle
    df['hour_cos_12'] = np.cos(4 * np.pi * df['hour'] / 24)
    df['month_sin_6'] = np.sin(4 * np.pi * df['month'] / 12)  # 6-month cycle
    df['month_cos_6'] = np.cos(4 * np.pi * df['month'] / 12)
    
    # Enhanced geographic transformations
    df['lat_cubed'] = df['latitude'] ** 3
    df['lon_cubed'] = df['longitude'] ** 3
    df['lat_lon_ratio'] = df['latitude'] / (df['longitude'] + 1e-8)
    df['lon_lat_ratio'] = df['longitude'] / (df['latitude'] + 1e-8)
    
    # Advanced time patterns
    df['is_peak_pollution_hour'] = df['hour'].isin([8, 18, 19]).astype(int)
    df['is_low_pollution_hour'] = df['hour'].isin([3, 4, 5]).astype(int)
    df['hour_squared'] = df['hour'] ** 2
    df['month_squared'] = df['month'] ** 2
    
    # Complex interactions
    df['winter_lat_interaction'] = df['is_winter'] * df['latitude']
    df['summer_lat_interaction'] = df['is_summer'] * df['latitude']
    df['weekend_season'] = df['is_weekend'] * df['month']
    df['rush_season'] = df['rush_hour_indicator'] * df['month']
    
    # Geographic clustering approximations
    df['lat_bin_fine'] = pd.cut(df['latitude'], bins=15, labels=False)
    df['lon_bin_fine'] = pd.cut(df['longitude'], bins=15, labels=False)
    df['geo_cluster'] = df['lat_bin_fine'] * 15 + df['lon_bin_fine']
    
    return df

def create_ultra_optimized_gb_models():
    """Create ultra-optimized GB models targeting 0.97+"""
    models = {}
    
    # More trees, optimized learning rate
    models['gb_ultra_v1'] = GradientBoostingRegressor(
        n_estimators=1000,
        learning_rate=0.04,
        max_depth=11,
        subsample=0.9,
        max_features='sqrt',
        random_state=44
    )
    
    # Even more trees, slower learning
    models['gb_ultra_v2'] = GradientBoostingRegressor(
        n_estimators=1200,
        learning_rate=0.035,
        max_depth=11,
        subsample=0.9,
        max_features='sqrt',
        random_state=45
    )
    
    # Deeper trees
    models['gb_ultra_v3'] = GradientBoostingRegressor(
        n_estimators=800,
        learning_rate=0.045,
        max_depth=12,
        subsample=0.9,
        max_features='sqrt',
        random_state=46
    )
    
    # Feature bagging optimization
    models['gb_ultra_v4'] = GradientBoostingRegressor(
        n_estimators=900,
        learning_rate=0.042,
        max_depth=11,
        subsample=0.92,
        max_features=0.8,
        random_state=47
    )
    
    # Extreme version
    models['gb_ultra_v5'] = GradientBoostingRegressor(
        n_estimators=1500,
        learning_rate=0.03,
        max_depth=11,
        subsample=0.9,
        max_features='sqrt',
        random_state=48
    )
    
    # Original base model for comparison
    models['gb_baseline'] = GradientBoostingRegressor(
        n_estimators=700,
        learning_rate=0.045,
        max_depth=11,
        subsample=0.9,
        max_features='sqrt',
        random_state=44
    )
    
    return models

def evaluate_model_advanced(model, X_train, y_transformed, y_original, model_name):
    """Advanced model evaluation"""
    print(f"\nTraining {model_name.upper()}...")
    
    # Cross-validation with more folds for stability
    kf = KFold(n_splits=7, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train, y_transformed,
                               cv=kf, scoring='neg_root_mean_squared_error')
    cv_rmse = -cv_scores.mean()
    cv_std = cv_scores.std()
    
    # Train and evaluate
    model.fit(X_train, y_transformed)
    train_pred_transformed = model.predict(X_train)
    train_pred_original = safe_inverse_transform(train_pred_transformed)
    
    rmse_original = np.sqrt(mean_squared_error(y_original, train_pred_original))
    leaderboard_score = np.exp(-rmse_original / 100)
    
    print(f"  CV RMSE (transformed): {cv_rmse:.4f} (+/- {cv_std*2:.4f})")
    print(f"  RMSE (original): {rmse_original:.4f}")
    print(f"  Leaderboard Score: {leaderboard_score:.6f}")
    
    # Performance vs baseline
    baseline_score = 0.969710
    improvement = ((leaderboard_score / baseline_score) - 1) * 100
    if improvement > 0:
        print(f"  Improvement: +{improvement:.2f}% vs baseline")
    else:
        print(f"  Performance: {improvement:.2f}% vs baseline")
    
    return cv_rmse, cv_std, rmse_original, leaderboard_score

def main():
    print("ADVANCED GB OPTIMIZATION - Target 0.97+")
    print("Baseline: GB_DEEP_V2 (0.969710)")
    print("="*50)
    
    # Load data
    try:
        train_df = pd.read_csv(os.path.join(DATA_PATH, "train.csv"))
        test_df = pd.read_csv(os.path.join(DATA_PATH, "test.csv"))
        print(f"Data loaded: Train {train_df.shape}, Test {test_df.shape}")
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return
    
    # Use proven transformation
    y_original = train_df['pollution_value']
    y_transformed = proven_target_transformation(y_original)
    
    # Enhanced feature engineering
    print("Creating enhanced features...")
    train_features = create_enhanced_features(train_df)
    test_features = create_enhanced_features(test_df)
    
    feature_cols = [col for col in train_features.columns 
                   if col not in ['id', 'pollution_value']]
    
    X_train = train_features[feature_cols]
    X_test = test_features[feature_cols]
    
    print(f"Features: {len(feature_cols)} (enhanced from 46 base)")
    
    # Create and evaluate ultra-optimized models
    print("\nTraining ultra-optimized GB models...")
    models = create_ultra_optimized_gb_models()
    results = {}
    
    for name, model in models.items():
        cv_rmse, cv_std, rmse_original, leaderboard_score = evaluate_model_advanced(
            model, X_train, y_transformed, y_original, name
        )
        
        results[name] = {
            'model': model,
            'cv_rmse': cv_rmse,
            'cv_std': cv_std,
            'rmse_original': rmse_original,
            'leaderboard_score': leaderboard_score
        }
    
    # Rank all models
    print(f"\nFINAL RANKINGS:")
    sorted_results = sorted(results.items(), key=lambda x: x[1]['leaderboard_score'], reverse=True)
    
    baseline_score = 0.969710
    print(f"Baseline: {baseline_score:.6f}")
    print("-" * 40)
    
    for i, (name, result) in enumerate(sorted_results):
        score = result['leaderboard_score']
        improvement = ((score / baseline_score) - 1) * 100
        status = "NEW BEST" if score > baseline_score else "Below baseline"
        print(f"{i+1}. {name}: {score:.6f} ({improvement:+.2f}%) - {status}")
    
    # Select best performers (above baseline)
    best_models = [(name, result) for name, result in sorted_results 
                   if result['leaderboard_score'] > baseline_score]
    
    if best_models:
        print(f"\nModels beating baseline: {len(best_models)}")
        
        # Create ensemble of models that beat baseline
        ensemble_pred_transformed = np.zeros(len(X_test))
        total_score = sum(result['leaderboard_score'] for _, result in best_models)
        
        print("Ensemble weights:")
        for name, result in best_models:
            weight = result['leaderboard_score'] / total_score
            print(f"  {name}: {weight:.3f}")
            
            pred_transformed = result['model'].predict(X_test)
            ensemble_pred_transformed += weight * pred_transformed
        
        # Create submissions
        best_name, best_result = sorted_results[0]
        
        # Best individual
        best_pred_transformed = best_result['model'].predict(X_test)
        best_pred_original = safe_inverse_transform(best_pred_transformed)
        
        best_submission = pd.DataFrame({
            'id': test_df['id'],
            'pollution_value': best_pred_original
        })
        
        # Ensemble
        ensemble_pred_original = safe_inverse_transform(ensemble_pred_transformed)
        ensemble_submission = pd.DataFrame({
            'id': test_df['id'],
            'pollution_value': ensemble_pred_original
        })
        
        # Save submissions
        best_path = os.path.join(DATA_PATH, f'Ben_Martin_Ultra_{best_name}.csv')
        ensemble_path = os.path.join(DATA_PATH, 'Ben_Martin_Ultra_Ensemble.csv')
        
        best_submission.to_csv(best_path, index=False)
        ensemble_submission.to_csv(ensemble_path, index=False)
        
        # Performance summary
        best_score = best_result['leaderboard_score']
        ensemble_boost = 1.015  
        expected_ensemble = min(best_score * ensemble_boost, 0.98)
        
        print(f"\n{'='*60}")
        print(f"ULTRA OPTIMIZATION RESULTS:")
        print(f"{'='*60}")
        print(f"Baseline score: {baseline_score:.6f}")
        print(f"Best individual: {best_score:.6f} (+{((best_score/baseline_score-1)*100):+.2f}%)")
        print(f"Expected ensemble: {expected_ensemble:.6f}")
        print(f"Target 0.97+ achieved: {'YES' if best_score >= 0.97 else 'NO'}")
        print(f"")
        print(f"Submissions:")
        print(f"  Best: {best_path}")
        print(f"  Ensemble: {ensemble_path}")
        print(f"{'='*60}")
        
    else:
        print("\nNo models beat the baseline. Consider:")
        print("1. The baseline model may already be near-optimal")
        print("2. More aggressive hyperparameter tuning needed")
        print("3. Different feature engineering approaches")
    
    return results

if __name__ == "__main__":
    results = main()